In [ ]:
#@title Install dependencies
%pip -q install pandas requests beautifulsoup4 trafilatura scikit-learn rapidfuzz pdfplumber tqdm

In [ ]:
#@title Imports
import os
import re
import io
import json
import math
import time
import csv
import hashlib
from collections import defaultdict, Counter
from dataclasses import dataclass
from urllib.parse import urljoin, urlparse

import pandas as pd
import requests
from bs4 import BeautifulSoup
import trafilatura
from tqdm import tqdm
from rapidfuzz import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pdfplumber

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Input parameters
base_url = input('Inserisci base_url (es. https://comune.vigone.to.it/): ').strip()
nome_comune = input('Inserisci nome_comune (opzionale ma consigliato): ').strip()
if not nome_comune:
    nome_comune = None

YEARS_TO_FILL = [2023, 2024]
ALLOW_EXTERNAL_OFFICIAL = input('ALLOW_EXTERNAL_OFFICIAL? [y/N]: ').strip().lower() in {'y', 'yes', 'true', '1'}

input_dir = '/content/drive/MyDrive/vigone_csv/'
output_dir = '/content/drive/MyDrive/vigone_output/'
os.makedirs(output_dir, exist_ok=True)

parsed_domain = urlparse(base_url).netloc
DOMAIN = parsed_domain
COMUNE = nome_comune or ''

In [ ]:
#@title Helpers: CSV parsing and missing detection
YEAR_RE = re.compile(r'20\d{2}')
MISSING_RE = re.compile(r'(inserire|\.\.\.|da compilare)', re.IGNORECASE)

SECTION_HINTS = [
    'Il Governo', 'Territorio e popolazione', 'I risultati in pillole', 'I servizi civici',
    'Rifiuti urbani', 'Progetti', 'Servizi', 'Personale', 'Patrimonio', 'Rendiconto'
]


def is_missing(val):
    if val is None:
        return True
    if isinstance(val, float) and math.isnan(val):
        return True
    if isinstance(val, str):
        if not val.strip():
            return True
        if MISSING_RE.search(val):
            return True
    return False


def detect_header_row(df, max_rows=10):
    best_row = 0
    best_count = -1
    for i in range(min(len(df), max_rows)):
        row = df.iloc[i].astype(str).tolist()
        count = sum(1 for cell in row if YEAR_RE.search(cell or ''))
        if count > best_count:
            best_count = count
            best_row = i
    return best_row


def get_year_columns(df):
    header_row = detect_header_row(df)
    header = df.iloc[header_row].astype(str).tolist()
    col_years = {}
    for idx, cell in enumerate(header):
        match = YEAR_RE.search(cell)
        if match:
            col_years[idx] = int(match.group(0))
    if not col_years:
        for idx in range(df.shape[1]):
            col_vals = df.iloc[:15, idx].astype(str).tolist()
            for val in col_vals:
                match = YEAR_RE.search(val)
                if match:
                    col_years[idx] = int(match.group(0))
                    break
    return col_years, header_row


def infer_row_label(row):
    for cell in row:
        text = str(cell).strip()
        if text and not YEAR_RE.search(text):
            return text
    return ''


def infer_section_context(df, row_idx):
    for i in range(row_idx - 1, max(-1, row_idx - 5), -1):
        row = df.iloc[i].astype(str).tolist()
        row_text = ' '.join([c.strip() for c in row if c.strip()])
        if not row_text:
            continue
        if any(h.lower() in row_text.lower() for h in SECTION_HINTS):
            return row_text
        if len(row_text) < 120 and sum(1 for c in row if c.strip()) <= 2:
            return row_text
    return ''

In [ ]:
#@title Helpers: crawling and text extraction
HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Safari/537.36'}


def fetch_url(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=15)
        if resp.status_code == 200:
            return resp.text, resp.headers.get('content-type', '')
    except requests.RequestException:
        return None, None
    return None, None


def extract_links(html, base):
    soup = BeautifulSoup(html, 'html.parser')
    links = set()
    for a in soup.find_all('a', href=True):
        href = urljoin(base, a['href'])
        links.add(href.split('#')[0])
    return links


def is_same_domain(url, domain):
    return urlparse(url).netloc == domain


def get_sitemaps(base_url):
    sitemap_urls = []
    robots_url = urljoin(base_url, '/robots.txt')
    text, _ = fetch_url(robots_url)
    if text:
        for line in text.splitlines():
            if line.lower().startswith('sitemap:'):
                sitemap_urls.append(line.split(':', 1)[1].strip())
    if not sitemap_urls:
        sitemap_urls.append(urljoin(base_url, '/sitemap.xml'))
    return sitemap_urls


def parse_sitemap(url):
    text, _ = fetch_url(url)
    if not text:
        return []
    soup = BeautifulSoup(text, 'xml')
    urls = [loc.text.strip() for loc in soup.find_all('loc') if loc.text]
    return urls


def crawl_site(base_url, domain, max_pages=400):
    visited = set()
    queue = [base_url]
    pages = []
    pdfs = set()
    while queue and len(visited) < max_pages:
        url = queue.pop(0)
        if url in visited or not is_same_domain(url, domain):
            continue
        visited.add(url)
        html, ctype = fetch_url(url)
        if not html:
            continue
        if 'pdf' in (ctype or '') or url.lower().endswith('.pdf'):
            pdfs.add(url)
            continue
        pages.append({'url': url, 'html': html})
        links = extract_links(html, url)
        for link in links:
            if link.lower().endswith('.pdf'):
                pdfs.add(link)
            elif is_same_domain(link, domain) and link not in visited:
                queue.append(link)
    return pages, list(pdfs)


def extract_text_from_html(html):
    downloaded = trafilatura.extract(html, include_comments=False, include_tables=True)
    if downloaded:
        return downloaded
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text('
')


def download_pdfs(pdf_urls, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    downloaded = []
    for url in tqdm(pdf_urls, desc='Downloading PDFs'):
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
            if resp.status_code == 200:
                fname = hashlib.md5(url.encode('utf-8')).hexdigest() + '.pdf'
                path = os.path.join(out_dir, fname)
                with open(path, 'wb') as f:
                    f.write(resp.content)
                downloaded.append((url, path))
        except requests.RequestException:
            continue
    return downloaded


def extract_text_from_pdf(path):
    texts = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or '')
    return '
'.join(texts)

In [ ]:
#@title Query Builder
SYNONYMS = {
    'delibera': ['delibera', 'deliberazione', 'deliberazioni'],
    'consiglio': ['consiglio comunale', 'c.c.', 'consiglio'],
    'giunta': ['giunta comunale', 'g.c.', 'giunta'],
    'odg': ['ordine del giorno', 'o.d.g.'],
    'rd': ['raccolta differenziata', 'rd'],
    'umido': ['umido', 'organico'],
    'multimateriale': ['multimateriale', 'vetro e lattine', 'vetro+lattine'],
    'indifferenziato': ['indifferenziato', 'secco residuo'],
}

INURL_VARIANTS = [
    'inurl:albo', 'inurl:atti', 'inurl:archivio', 'inurl:trasparenza'
]

FILETYPE_VARIANTS = [
    'filetype:pdf', 'filetype:doc', 'filetype:docx', 'filetype:xls', 'filetype:xlsx'
]

QUERY_TEMPLATES = {
    'DELIBERE_CC': [
        'site:{DOMAIN} "VERBALE DI DELIBERAZIONE" "DEL CONSIGLIO COMUNALE" {YEAR}',
        'site:{DOMAIN} "Deliberazione C.C." ("n." OR "N.") {YEAR} {COMUNE}',
        'site:{DOMAIN} "L'anno duemilaventitré" "DEL CONSIGLIO COMUNALE" "N."',
        'site:{DOMAIN} "L'anno duemilaventiquattro" "DEL CONSIGLIO COMUNALE" "N."',
    ],
    'DELIBERE_GC': [
        'site:{DOMAIN} "VERBALE DI DELIBERAZIONE" "DELLA GIUNTA COMUNALE" {YEAR}',
        'site:{DOMAIN} "Deliberazione G.C." ("n." OR "N.") {YEAR} {COMUNE}',
        'site:{DOMAIN} "L'anno duemilaventitré" "DELLA GIUNTA COMUNALE" "N."',
        'site:{DOMAIN} "L'anno duemilaventiquattro" "DELLA GIUNTA COMUNALE" "N."',
    ],
    'SEDUTE_CC': [
        'site:{DOMAIN} "CONVOCAZIONE" "CONSIGLIO COMUNALE" {YEAR}',
        'site:{DOMAIN} ("ORDINE DEL GIORNO" OR "O.D.G.") "CONSIGLIO COMUNALE" {YEAR}',
    ],
    'SEDUTE_GC': [
        'site:{DOMAIN} "CONVOCAZIONE" "GIUNTA COMUNALE" {YEAR}',
        'site:{DOMAIN} ("ORDINE DEL GIORNO" OR "O.D.G.") "GIUNTA COMUNALE" {YEAR}',
    ],
    'PERSONALE': [
        'site:{DOMAIN} "personale in servizio" ("31 dicembre {YEAR}" OR "{YEAR}")',
        'site:{DOMAIN} ("Relazione sulla performance" OR "PIAO" OR "Piano Integrato") {YEAR} personale',
        'site:{DOMAIN} ("Conto annuale del personale" OR "dotazione organica" OR "fabbisogni") {YEAR}',
    ],
    'ORGANIGRAMMA': [
        'site:{DOMAIN} (organigramma OR funzionigramma OR "articolazione degli uffici")',
        'site:{DOMAIN} inurl:amministrazione (ufficio OR area OR settore)',
        'site:{DOMAIN} ("Area Tecnica" OR "Area Amministrativa") "Personale"',
    ],
    'SERVIZIO_CIVILE': [
        'site:{DOMAIN} "servizio civile" (volontari OR operatori OR "Servizio Civile Universale") {YEAR}',
        'site:{DOMAIN} (bando OR graduatoria) "servizio civile" {YEAR}',
        'site:{DOMAIN} "Servizio Civile Universale" {COMUNE}',
    ],
    'PERSONALE_FASCE': [
        'site:{DOMAIN} (personale OR dipendenti) (genere OR maschi OR femmine) ("fasce di età" OR "anni")',
        'site:{DOMAIN} filetype:pdf ("conto annuale" OR PIAO) (età OR fascia) {YEAR}',
    ],
    'DEMOGRAFIA': [
        'site:{DOMAIN} "Popolazione residente" {YEAR} {COMUNE}',
        'site:{DOMAIN} "Bilancio demografico" {YEAR} {COMUNE}',
        'site:{DOMAIN} (nati OR morti OR popolazione) {YEAR} {COMUNE}',
        'site:dati.istat.it "{COMUNE}" "Popolazione residente" {YEAR}',
        'site:demo.istat.it "{COMUNE}" ("bilancio demografico" OR "popolazione") {YEAR}',
        'site:dati.istat.it "{COMUNE}" (Nati OR Morti) {YEAR}',
    ],
    'PATRIMONIO_NETTO': [
        'site:{DOMAIN} (rendiconto OR "stato patrimoniale") filetype:pdf {YEAR} ("Patrimonio netto" OR "Ricchezza netta")',
    ],
    'DEBITI': [
        'site:{DOMAIN} "stato patrimoniale" filetype:pdf {YEAR} Debiti',
    ],
    'RISULTATO_ECONOMICO': [
        'site:{DOMAIN} "conto economico" filetype:pdf {YEAR} ("Risultato d'esercizio" OR "Risultato economico")',
    ],
    'MISSIONE_INVESTIMENTI': [
        'site:{DOMAIN} rendiconto filetype:pdf {YEAR} ("Missione 09" OR "MISSIONE 9") ("conto capitale" OR investimenti)',
        'site:{DOMAIN} rendiconto filetype:pdf {YEAR} ("Missione 04" OR "MISSIONE 4") ("conto capitale" OR investimenti)',
        'site:{DOMAIN} rendiconto filetype:pdf {YEAR} ("Missione 12" OR "MISSIONE 12") ("conto capitale" OR investimenti)',
        'site:{DOMAIN} rendiconto filetype:pdf {YEAR} ("Missione 05" OR "MISSIONE 5") ("conto capitale" OR investimenti)',
        'site:{DOMAIN} rendiconto filetype:pdf {YEAR} ("Missione 06" OR "MISSIONE 6") ("conto capitale" OR investimenti)',
    ],
    'BDAP_MISSIONI': [
        'site:bdap-opendata.mef.gov.it {COMUNE} {YEAR} missione spesa',
        'site:bdap-opendata.mef.gov.it {COMUNE} {YEAR} rendiconto',
    ],
    'ALIQUOTE_IMU': [
        'site:finanze.gov.it "aliquote IMU" {COMUNE} {YEAR}',
        'site:{DOMAIN} "aliquote IMU" {YEAR}',
    ],
    'ADDIZIONALE_IRPEF': [
        'site:finanze.gov.it "addizionale comunale" "IRPEF" {COMUNE} {YEAR}',
        'site:{DOMAIN} "addizionale comunale IRPEF" {YEAR}',
    ],
    'SOCIAL': [
        'site:{DOMAIN} (facebook OR instagram OR youtube OR telegram) {COMUNE}',
        'site:{DOMAIN} "Facebook" {COMUNE}',
        'site:{DOMAIN} "Instagram" {COMUNE}',
    ],
    'SERVIZI_CIVICI': [
        'site:{DOMAIN} ("polizia locale" OR "comando") {YEAR} (relazione OR report) filetype:pdf',
        'site:{DOMAIN} "art. 208" {YEAR} filetype:pdf',
        'site:{DOMAIN} (CILA OR SCIA OR "permesso di costruire" OR PDC) {YEAR}',
        'site:{DOMAIN} (manutenzione OR "verde pubblico" OR potatura OR disinfestazione) {YEAR} filetype:pdf',
        'site:{DOMAIN} biblioteca {YEAR} (MLOL OR ebook OR accessi)',
    ],
    'RIFIUTI': [
        'site:{DOMAIN} {YEAR} (rifiuti OR "igiene urbana") ("raccolta differenziata" OR RD) (kg OR tonnellate OR "%") filetype:pdf',
    ],
    'RIFIUTI_EXTERNAL': [
        'site:catasto-rifiuti.isprambiente.it "{COMUNE}" {YEAR} "raccolta differenziata"',
        'site:isprambiente.gov.it filetype:pdf "Rapporto rifiuti urbani" {YEAR} {COMUNE}',
    ],
    'PROGETTI': [
        'site:{DOMAIN} (PNRR OR "Italia Domani") (CUP OR "Codice Unico di Progetto") filetype:pdf',
        'site:{DOMAIN} ("Programma triennale" OR DUP) {YEAR} (opere OR interventi OR investimenti) filetype:pdf',
    ],
    'GENERIC': [
        'site:{DOMAIN} {COMUNE} {YEAR} {LABEL}',
        'site:{DOMAIN} {LABEL} {YEAR} filetype:pdf',
    ]
}


def categorize_cell(row_label, section_context):
    label = (row_label or '').lower()
    context = (section_context or '').lower()
    if 'imu' in label or 'aliquota' in label:
        return 'ALIQUOTE_IMU'
    if 'irpef' in label or 'addizionale' in label:
        return 'ADDIZIONALE_IRPEF'
    if 'delib' in label and 'consiglio' in label:
        return 'DELIBERE_CC'
    if 'delib' in label and 'giunta' in label:
        return 'DELIBERE_GC'
    if 'sedut' in label and 'consiglio' in label:
        return 'SEDUTE_CC'
    if 'sedut' in label and 'giunta' in label:
        return 'SEDUTE_GC'
    if 'personale' in label and ('età' in label or 'fasce' in label):
        return 'PERSONALE_FASCE'
    if 'personale' in label or 'dipend' in label:
        return 'PERSONALE'
    if 'organigramma' in label or 'uffici' in label or 'struttura' in label:
        return 'ORGANIGRAMMA'
    if 'servizio civile' in label:
        return 'SERVIZIO_CIVILE'
    if 'popolazione' in label or 'nati' in label or 'morti' in label or 'demograf' in label:
        return 'DEMOGRAFIA'
    if 'patrimonio' in label:
        return 'PATRIMONIO_NETTO'
    if 'debiti' in label:
        return 'DEBITI'
    if 'risultato' in label and 'econom' in label:
        return 'RISULTATO_ECONOMICO'
    if 'missione' in label or 'investimenti' in label:
        return 'MISSIONE_INVESTIMENTI'
    if 'facebook' in label or 'instagram' in label or 'youtube' in label or 'social' in label:
        return 'SOCIAL'
    if 'rifiuti' in label or 'raccolta' in label or 'rd' in label:
        return 'RIFIUTI'
    if 'progetto' in label or 'pnrr' in label or 'cup' in label:
        return 'PROGETTI'
    if 'polizia' in label or 'cila' in label or 'scia' in label or 'biblioteca' in label:
        return 'SERVIZI_CIVICI'
    if 'territorio' in context or 'popolazione' in context:
        return 'DEMOGRAFIA'
    return 'GENERIC'


def build_queries(category, domain, comune, year, row_label, allow_external):
    templates = list(QUERY_TEMPLATES.get(category, []))
    if category == 'RIFIUTI' and allow_external:
        templates.extend(QUERY_TEMPLATES.get('RIFIUTI_EXTERNAL', []))
    if category == 'DEMOGRAFIA' and not allow_external:
        templates = [t for t in templates if '{DOMAIN}' in t]
    if category in {'ALIQUOTE_IMU', 'ADDIZIONALE_IRPEF', 'BDAP_MISSIONI'} and not allow_external:
        templates = [t for t in templates if '{DOMAIN}' in t]
    if category == 'GENERIC':
        templates = templates + [
            'site:{DOMAIN} "{LABEL}" {YEAR} {COMUNE}',
            'site:{DOMAIN} "{LABEL}" {YEAR} ' + ' OR '.join(INURL_VARIANTS),
        ]
    if year != 2023:
        templates = [t for t in templates if 'duemilaventitré' not in t]
    if year != 2024:
        templates = [t for t in templates if 'duemilaventiquattro' not in t]
    expanded = []
    for template in templates:
        expanded.append(template)
        if 'site:{DOMAIN}' in template:
            for inurl in INURL_VARIANTS:
                expanded.append(f"{template} {inurl}")
            for ftype in FILETYPE_VARIANTS:
                expanded.append(f"{template} {ftype}")
    formatted = []
    for q in expanded:
        formatted.append(q.format(DOMAIN=domain, YEAR=year, COMUNE=comune or '', LABEL=row_label))
    unique = []
    seen = set()
    for q in formatted:
        q = ' '.join(q.split())
        if q and q not in seen:
            seen.add(q)
            unique.append(q)
    return unique


def query_priority(query):
    score = 1
    if 'filetype:pdf' in query:
        score += 2
    if 'rendiconto' in query or 'stato patrimoniale' in query or 'verbale di deliberazione' in query:
        score += 2
    return score


def query_to_text(query):
    cleaned = re.sub(r'(site:|filetype:|inurl:)\S+', '', query)
    cleaned = cleaned.replace('OR', ' ')
    cleaned = cleaned.replace('(', ' ').replace(')', ' ')
    cleaned = cleaned.replace('"', ' ')
    return ' '.join(cleaned.split())

In [ ]:
#@title Build corpus and index
print('Crawling domain:', DOMAIN)

all_urls = set()
for sitemap in get_sitemaps(base_url):
    all_urls.update(parse_sitemap(sitemap))

pages, pdfs = crawl_site(base_url, DOMAIN, max_pages=400)

all_urls.update([p['url'] for p in pages])
all_urls.update(pdfs)

print('Pages:', len(pages))
print('PDFs:', len(pdfs))

text_docs = []
for page in tqdm(pages, desc='Extracting HTML'):
    text = extract_text_from_html(page['html'])
    if text:
        text_docs.append({'url': page['url'], 'text': text})

pdf_dir = os.path.join(output_dir, 'pdf_cache')
pdf_files = download_pdfs(pdfs, pdf_dir)
for url, path in tqdm(pdf_files, desc='Extracting PDFs'):
    try:
        text = extract_text_from_pdf(path)
        if text:
            text_docs.append({'url': url, 'text': text})
    except Exception:
        continue

print('Documents indexed:', len(text_docs))

vectorizer = TfidfVectorizer(stop_words='italian', max_features=50000)
corpus = [doc['text'] for doc in text_docs]
if corpus:
    tfidf_matrix = vectorizer.fit_transform(corpus)
else:
    tfidf_matrix = None

In [ ]:
#@title Retrieval and extraction
NUM_RE = re.compile(r'(\d+[\.,]?\d*)\s?(€|euro|%|kg|tonnellate|t)?', re.IGNORECASE)


def rerank_docs(candidate_indices, row_label, year):
    scored = []
    label = (row_label or '').lower()
    for idx, score in candidate_indices:
        text = text_docs[idx]['text'].lower()
        bonus = 0
        if str(year) in text:
            bonus += 0.2
        if label and label in text:
            bonus += 0.2
        scored.append((idx, score + bonus))
    scored.sort(key=lambda x: x[1], reverse=True)
    return scored


def retrieve_docs(queries, top_k=5):
    if tfidf_matrix is None:
        return []
    best_scores = defaultdict(float)
    for q in queries:
        q_text = query_to_text(q)
        if not q_text:
            continue
        q_vec = vectorizer.transform([q_text])
        scores = linear_kernel(q_vec, tfidf_matrix).flatten()
        top_indices = scores.argsort()[::-1][:top_k]
        for idx in top_indices:
            best_scores[idx] = max(best_scores[idx], scores[idx])
    ranked = sorted(best_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked


def extract_value_from_text(text, row_label, year):
    label = (row_label or '').lower()
    lines = text.split('
')
    candidates = []
    for line in lines:
        if str(year) in line or label in line.lower():
            for match in NUM_RE.finditer(line):
                num = match.group(1)
                unit = match.group(2) or ''
                snippet = line.strip()
                candidates.append((num, unit, snippet))
    if candidates:
        return candidates[0]
    for match in NUM_RE.finditer(text):
        return match.group(1), match.group(2) or '', text[:200]
    return None

In [ ]:
#@title Process CSVs and fill values
input_files = [f for f in os.listdir(input_dir) if f.lower().endswith('.csv')]

queries_audit_path = os.path.join(output_dir, 'queries_generated.csv')
sources_path = os.path.join(output_dir, 'sources_long.csv')
report_path = os.path.join(output_dir, 'run_report.md')

query_rows = []
source_rows = []
report_lines = []

for fname in input_files:
    fpath = os.path.join(input_dir, fname)
    df_raw = pd.read_csv(fpath, header=None, dtype=str, keep_default_na=False)
    col_years, header_row = get_year_columns(df_raw)

    df_filled = df_raw.copy()
    filled_count = 0
    total_targets = 0

    for row_idx in range(header_row + 1, len(df_raw)):
        row = df_raw.iloc[row_idx].tolist()
        row_label = infer_row_label(row)
        section_context = infer_section_context(df_raw, row_idx)
        if not row_label:
            continue
        category = categorize_cell(row_label, section_context)

        for col_idx, year in col_years.items():
            if year not in YEARS_TO_FILL:
                continue
            cell_val = df_raw.iat[row_idx, col_idx]
            if not is_missing(cell_val):
                continue
            total_targets += 1
            queries = build_queries(category, DOMAIN, COMUNE, year, row_label, ALLOW_EXTERNAL_OFFICIAL)
            for q in queries:
                query_rows.append({
                    'input_file': fname,
                    'section': section_context,
                    'row_label': row_label,
                    'col_year': year,
                    'query': q,
                    'priority': query_priority(q),
                    'notes': category
                })
            ranked = retrieve_docs(queries, top_k=8)
            ranked = rerank_docs(ranked, row_label, year)
            value = None
            if ranked:
                top_idx = ranked[0][0]
                doc = text_docs[top_idx]
                extracted = extract_value_from_text(doc['text'], row_label, year)
                if extracted:
                    num, unit, snippet = extracted
                    value = f"{num} {unit}".strip()
                    source_rows.append({
                        'input_file': fname,
                        'row_label': row_label,
                        'section': section_context,
                        'year': year,
                        'value': value,
                        'source_url': doc['url'],
                        'snippet': snippet[:400]
                    })
            if value:
                df_filled.iat[row_idx, col_idx] = value
                filled_count += 1
            else:
                source_rows.append({
                    'input_file': fname,
                    'row_label': row_label,
                    'section': section_context,
                    'year': year,
                    'value': '',
                    'source_url': '',
                    'snippet': 'NOT_FOUND'
                })

    output_file = os.path.join(output_dir, fname.replace('.csv', '_filled.csv'))
    df_filled.to_csv(output_file, header=False, index=False)
    report_lines.append(f"## {fname}")
    report_lines.append(f"Targets: {total_targets}, Filled: {filled_count}")

if query_rows:
    pd.DataFrame(query_rows).to_csv(queries_audit_path, index=False)
if source_rows:
    pd.DataFrame(source_rows).to_csv(sources_path, index=False)

with open(report_path, 'w', encoding='utf-8') as f:
    f.write('
'.join(report_lines))

print('Done. Outputs saved to', output_dir)